In [27]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('once')
warnings.filterwarnings('ignore', category=RuntimeWarning)
import itertools
from datetime import datetime, timedelta
import optuna
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.regression.linear_model import WLS
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from sklearn.utils import resample
from joblib import Parallel, delayed
from datetime import datetime, timedelta
import traceback
import os
import pickle
from collections import defaultdict

"""
LOAD DATA FROM CSV FILE 
"""
def load_file():    
    # Path to your CSV file
    file_path = 'Optiver Assignment - Sheet1.csv'

    # Read the CSV file, use the third row as header, and skip the first column
    raw_df = pd.read_csv(file_path, header=2, usecols=lambda x: x != 'Unnamed: 0')

    # Rename the first column to 'Date' and convert to datetime format
    raw_df.rename(columns={raw_df.columns[0]: 'Date'}, inplace=True)
    raw_df['Date'] = pd.to_datetime(raw_df['Date'], dayfirst=True)

    # Set 'Date' as the index of the DataFrame
    raw_df.set_index('Date', inplace=True)

    return raw_df

raw_df = load_file()

"""
CONFIRM DATA LOADED CORRECTLY
"""
# Verify the conversion by displaying the data types and first few rows

if not isinstance(raw_df.index, pd.DatetimeIndex):
    print("Index is not a DateTimeIndex. Attempting to convert...")
    raw_df.index = pd.to_datetime(raw_df.index)
    
raw_df.head(2)

,ESX_realized_1,ESX_realized_5,ESX_realized_10,ESX_realized_50,ESX_realized_100,ESX_move_1,ESX_move_5,ESX_move_10,ESX_move_50,ESX_move_100,...,ESPX_atm_5,ESPX_atm_25,ESPX_atm_50,GBL_atm_5,GBL_atm_25,GBL_atm_50,TN10_atm_5,TN10_atm_25,TN10_atm_50,ESX_atm_50
Date,,,,,,,,,,,,,,,,,,,,,
2014-05-27 09:30:00,0.083026,0.107824,0.119397,0.151017,0.155044,0.004350,0.021941,0.010284,0.070762,0.053985,...,0.079740,0.098468,0.111800,0.048067,0.047586,0.046379,0.039570,0.043525,0.043889,0.140849
2014-05-27 09:45:00,0.079815,0.106227,0.119300,0.150970,0.155031,0.003105,0.019409,0.011224,0.069462,0.054625,...,0.079618,0.098372,0.111722,0.048067,0.047586,0.046379,0.039592,0.043707,0.043890,0.140799


In [2]:
"""
NG: I removed consecutive NaN in data and use spline interpolation to fill cases where there were not
excessive NaNs
"""

def handle_missing_data(df, consecutive_nan_threshold=3):
    """
    Handles missing data in a DataFrame by removing rows with consecutive NaNs beyond a threshold and filling the remaining NaNs.

    Parameters:
    df (pd.DataFrame): The DataFrame to be cleaned.
    consecutive_nan_threshold (int, optional): The threshold for consecutive NaNs. Rows with NaNs exceeding this count will be dropped.

    Returns:
    pd.DataFrame: The cleaned DataFrame with missing data handled.

    """    
    def drop_consecutive_nans(df):
        """
        Drops rows from the DataFrame where any column has consecutive NaNs exceeding the specified threshold.

        Parameters:
        df (pd.DataFrame): The DataFrame from which rows are to be dropped.

        Returns:
        pd.DataFrame: The DataFrame after dropping rows.

        """        
        rows_to_remove = set()
        for col in df.columns:
            is_nan = df[col].isna()
            filled = is_nan != is_nan.shift()
            cumsum = filled.cumsum()
            consecutive_nans = is_nan.groupby(cumsum).cumsum()
            rows_to_remove.update(consecutive_nans[consecutive_nans > consecutive_nan_threshold].index)
        return df.drop(rows_to_remove, axis=0)
    
    def fill_nans(df):
        """
        Fills NaN values in the DataFrame. Numeric columns are filled using linear interpolation, and others using forward and backward fill.

        Parameters:
        df (pd.DataFrame): The DataFrame where NaNs are to be filled.

        Returns:
        pd.DataFrame: The DataFrame after filling NaNs.

        """        
        for col in df.columns:
            if pd.api.types.is_numeric_dtype(df[col]):
                # Using linear interpolation as a simpler alternative
                df[col].interpolate(method='linear', inplace=True, limit_direction='both', limit_area='inside')
            df[col].bfill(inplace=True)
            df[col].ffill(inplace=True)
        return df

    cleaned_df = df.copy()
    cleaned_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    if df.index.name != "Date" or not isinstance(df.index, pd.DatetimeIndex):
        cleaned_df.index = pd.to_datetime(cleaned_df.index, errors='coerce')

    cleaned_df = drop_consecutive_nans(cleaned_df)
    cleaned_df = fill_nans(cleaned_df)

    return cleaned_df

# Assuming df is your original DataFrame
interpolated_df = handle_missing_data(raw_df)
interpolated_df.head(2)

,ESX_realized_1,ESX_realized_5,ESX_realized_10,ESX_realized_50,ESX_realized_100,ESX_move_1,ESX_move_5,ESX_move_10,ESX_move_50,ESX_move_100,...,ESPX_atm_5,ESPX_atm_25,ESPX_atm_50,GBL_atm_5,GBL_atm_25,GBL_atm_50,TN10_atm_5,TN10_atm_25,TN10_atm_50,ESX_atm_50
Date,,,,,,,,,,,,,,,,,,,,,
2014-05-27 09:30:00,0.083026,0.107824,0.119397,0.151017,0.155044,0.004350,0.021941,0.010284,0.070762,0.053985,...,0.079740,0.098468,0.111800,0.048067,0.047586,0.046379,0.039570,0.043525,0.043889,0.140849
2014-05-27 09:45:00,0.079815,0.106227,0.119300,0.150970,0.155031,0.003105,0.019409,0.011224,0.069462,0.054625,...,0.079618,0.098372,0.111722,0.048067,0.047586,0.046379,0.039592,0.043707,0.043890,0.140799


In [3]:
"""
NG: My prior is that ESPX volatility has a beta to ESX volatility and that this was the most
explanatory predictor. The reason why I was late in submitting was that I was very shocked
that most of my results in my work here were tenous except for this result that I found in the first 15
minutes, which led me to be both curious and determined to find other factors that would predict
residual changes in implied volatility after the ESPX component was removed. To reduce multi-collinearity
I also transformed the other predictors to idiosyncratic volatility via the same method. 
"""
def transform_df(df):
    
    # Calculates rolling beta
    def calculate_rolling_beta(df, window=300):
        cov = df['ESX_move_1'].rolling(window).cov(df['ESPX_move_1'])
        var = df['ESPX_move_1'].rolling(window).var()
        df['ESX_daily_beta_to_ESPX'] = cov / var
        return df

    # Helper function to calculate idiosyncratic variance
    def calculate_idio_variance_for_row(row, total_volatility_responder, total_volatility_predictor, beta_to_ESPX):
        variance_responder = row[total_volatility_responder] ** 2
        variance_predictor = row[total_volatility_predictor] ** 2
        idiosyncratic_variance = variance_responder - (beta_to_ESPX ** 2 * variance_predictor)
        return idiosyncratic_variance

    # Calculates idiosyncratic variance
    def calculate_idio_variances(df, data_on_assets):
        temp_df = df.copy()
        esx_beta_col = 'ESX_daily_beta_to_ESPX'
        temp_df['ESX_atm_50_idio_var'] = temp_df.apply(lambda row: calculate_idio_variance_for_row(row, 'ESX_atm_50', 'ESPX_atm_50', row[esx_beta_col]), axis=1)

        for data_on_asset in data_on_assets:
            asset_name = data_on_asset['asset_name']
            beta_to_ESPX = data_on_asset['total_return_beta_to_ESPX']
            idio_var_col_name = f'{asset_name}_idio_var'
            temp_df[idio_var_col_name] = temp_df.apply(lambda row: calculate_idio_variance_for_row(row, asset_name, 'ESPX_atm_50', beta_to_ESPX), axis=1)
        return temp_df
    
    # Calculates ratios to atm_50
    def calculate_atm_ratios(df):
        temp_df = df.copy()
        for col in temp_df.columns:
            if 'atm_5' in col or 'atm_25' in col:
                asset = col.split('_')[0]
                temp_df[f'{col}_ratio_to_atm_50_idio_var'] = temp_df[col] / temp_df[f'{asset}_atm_50_idio_var']
        return temp_df

    # Calculates ratios relative to ESX_move_50
    def calculate_move_ratios(df):
        temp_df = df.copy()
        for col in temp_df.columns:
            if 'ESX_move_' in col and col != 'ESX_move_50' and 'ratio' not in col:
                asset = col.split('_')[0]            
                temp_df[f'{col}_ratio_to_move_50'] = temp_df[col] / temp_df[f'{asset}_move_50']
        return temp_df

    # Calculates ratios relative to corresponding ESPX_move
    def calculate_esx_espx_ratios(df):
        temp_df = df.copy()
        for esx_col in temp_df.columns:
            if 'ESX_move_' in esx_col and '_ratio' not in esx_col:
                espx_col = esx_col.replace('ESX', 'ESPX')
                temp_df[f'{esx_col}_ratio_to_{espx_col}'] = temp_df[esx_col] / temp_df[espx_col]
        return temp_df
    
    beta_assumptions = {'TN10': 0.3, 'GBL': 0.2}
    data_on_assets = []
    
    df_transformed_1A = calculate_rolling_beta(interpolated_df)
    
    # Helper function for assigning beta assumptions
    def assign_beta_assumptions(column_name, beta_assumptions):
        for key in beta_assumptions.keys():
            if key in column_name:
                return beta_assumptions[key]
        return None  # Default case if no match is found

    for column in df_transformed_1A.columns:
        if '_atm_' in column:
            beta_value = assign_beta_assumptions(column, beta_assumptions)
            if beta_value is not None:  # Ensure that a beta value was assigned
                data_on_assets.append({
                    'asset_name': column,
                    'total_return_beta_to_ESPX': beta_value
                })
    
    # Series of pandas transformations
    df_transformed_1B = calculate_idio_variances(df_transformed_1A, data_on_assets)
    important_columns = ['ESX_atm_50'] + [col for col in df_transformed_1B.columns if '_idio_var' in col or '_move_' in col]
    df_transformed_2 = df_transformed_1B[important_columns]
    df_transformed_3 = calculate_atm_ratios(df_transformed_2)
    df_transformed_4 = calculate_move_ratios(df_transformed_3)
    df_transformed_5 = calculate_esx_espx_ratios(df_transformed_4)
    unimportant_columns = [col for col in df_transformed_5.columns if 'move_' in col and '_ratio_to_' not in col]
    df_transformed_6 = df_transformed_5.drop(columns=unimportant_columns)
    return df_transformed_6
    
clean_df = transform_df(interpolated_df)
clean_df.head(2)

,ESX_atm_50,ESX_atm_50_idio_var,GBL_atm_5_idio_var,GBL_atm_25_idio_var,GBL_atm_50_idio_var,TN10_atm_5_idio_var,TN10_atm_25_idio_var,TN10_atm_50_idio_var,ESX_atm_50_ratio_to_atm_50_idio_var,ESX_atm_50_idio_var_ratio_to_atm_50_idio_var,...,TN10_atm_50_idio_var_ratio_to_atm_50_idio_var,ESX_move_1_ratio_to_move_50,ESX_move_5_ratio_to_move_50,ESX_move_10_ratio_to_move_50,ESX_move_100_ratio_to_move_50,ESX_move_1_ratio_to_ESPX_move_1,ESX_move_5_ratio_to_ESPX_move_5,ESX_move_10_ratio_to_ESPX_move_10,ESX_move_50_ratio_to_ESPX_move_50,ESX_move_100_ratio_to_ESPX_move_100
Date,,,,,,,,,,,,,,,,,,,,,
2014-05-27 09:30:00,0.140849,NaN,0.001810,0.001764,0.001651,0.000441,0.000769,0.000801,NaN,NaN,...,1.0,0.061472,0.310067,0.145327,0.762906,1.377182,1.908343,1.592074,2.246363,1.293474
2014-05-27 09:45:00,0.140799,NaN,0.001811,0.001765,0.001652,0.000444,0.000787,0.000803,NaN,NaN,...,1.0,0.044703,0.279420,0.161583,0.786406,0.983090,1.707858,1.851313,2.038159,1.308810


In [4]:
"""
NG: I performed additional transformations such as clamping outliers and converting to first differences
to reduce heteroskedasticity make the data better conform to the assumptions of OLS. Even after
performing these transformations the data was heteroskedastic. The data is also autocorrelated because
I used overlapping time periods. 
"""

def clamp_columns(df, exclude_columns=None):
    """
    Clamps the values of each column in the DataFrame within the 1st and 99th percentile,
    excluding specified columns.

    Parameters:
    df (pd.DataFrame): The DataFrame to be clamped.
    exclude_columns (list, optional): List of column names to be excluded from clamping.

    Returns:
    pd.DataFrame: DataFrame with clamped column values.
    """    
    if exclude_columns is None:
        exclude_columns = ['Date']

    # Drop duplicate indices once before the loop
    df = df[~df.index.duplicated(keep='first')]
    clamped_df = df.copy()
    
    for col in df.columns:
        if col not in exclude_columns:
            lower_limit = df[col].quantile(0.01)
            upper_limit = df[col].quantile(0.99)
            clamped_df[col] = df[col].clip(lower=lower_limit, upper=upper_limit)
    
    return clamped_df

def add_column_of_diffs(df, shift_period, exclude_columns=None):
    """
    Adds columns to the DataFrame representing the difference from a shifted period,
    excluding specified columns.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    shift_period (int): The number of periods to shift for calculating differences.
    exclude_columns (list, optional): List of column names to be excluded.

    Returns:
    pd.DataFrame: DataFrame with additional columns showing differences.
    """    
    add_column_of_diffs_df_copy = df.copy()
#     print("df describer",df.describe)
    if exclude_columns is None:
        exclude_columns = []

    for col in add_column_of_diffs_df_copy.columns:
        if col not in exclude_columns:
            diff_col_name = f"{col}_shift_{shift_period}"
            if diff_col_name not in add_column_of_diffs_df_copy.columns:
                add_column_of_diffs_df_copy[diff_col_name] = add_column_of_diffs_df_copy[col] / add_column_of_diffs_df_copy[col].shift(shift_period) - 1

    return add_column_of_diffs_df_copy

def calculate_rolling_z_scores(df, window='40D', exclude_columns=None):
    """
    Calculates the rolling Z-scores for each column in the DataFrame over a specified window,
    excluding certain columns.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    window (str): The rolling window size.
    exclude_columns (list, optional): List of column names to be excluded.

    Returns:
    pd.DataFrame: DataFrame containing rolling Z-scores.
    """    
    calculate_rolling_z_scores_df_copy = df.copy()
    if exclude_columns is None:
        exclude_columns = ['OLS_residuals']

    z_score_series = []
    for col in calculate_rolling_z_scores_df_copy.columns:
        if col not in exclude_columns:
            non_nan_series = calculate_rolling_z_scores_df_copy[col].dropna()
            rolling_mean = non_nan_series.rolling(window=window, min_periods=1).mean()
            rolling_std = non_nan_series.rolling(window=window, min_periods=1).std()            
            z_score_series.append((calculate_rolling_z_scores_df_copy[col] - rolling_mean) / rolling_std)

    z_scores_df = pd.concat(z_score_series, axis=1)
    z_scores_df.columns = [f"z_score_{col}" for col in df.columns if col not in exclude_columns]
    z_scores_df.index = calculate_rolling_z_scores_df_copy.index
    return z_scores_df

def remove_duplicate_columns(df):
    """
    Removes duplicate columns from a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame from which duplicate columns are to be removed.

    Returns:
    pd.DataFrame: DataFrame with duplicate columns removed.
    """    
    duplicates = []
    columns = df.columns.tolist()
    for i in range(len(columns)):
        if columns[i] not in duplicates:
            for j in range(i+1, len(columns)):
                if df[columns[i]].equals(df[columns[j]]):
                    duplicates.append(columns[j])
    return df.drop(columns=duplicates)

def preprocess_data(df, window_days, shift_period, end_time_input):
    """
    Preprocesses the data by calculating rolling Z-scores, adding shifted difference columns,
    removing duplicate columns, and filtering data based on a specified time window.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    window_days (int): Number of days for the rolling window for Z-scores.
    shift_period (int): Period for shifted difference calculations.
    end_time_input (str or pd.Timestamp or datetime.time): End time for filtering data.

    Returns:
    pd.DataFrame: The preprocessed DataFrame.  
    """
    preprocess_data_df_copy = df.copy()
    window = f"{window_days}D"
    df_w_rolling_z_scores = calculate_rolling_z_scores(preprocess_data_df_copy, window=window, exclude_columns=None)

    na_count = df_w_rolling_z_scores.isna().sum()
    df_w_rolling_z_scores_clean = df_w_rolling_z_scores.loc[:, na_count <= 1000].dropna()

    df_w_rolling_z_scores_clean_w_diffs = add_column_of_diffs(df_w_rolling_z_scores_clean, shift_period, exclude_columns=['TimeOfDay', 'level_0'])
    df_w_rolling_z_scores_clean_w_diffs = df_w_rolling_z_scores_clean_w_diffs.replace([np.inf, -np.inf], np.nan)
    
    df_w_rolling_z_scores_clean_w_diffs_deduped = remove_duplicate_columns(df_w_rolling_z_scores_clean_w_diffs[~df_w_rolling_z_scores_clean_w_diffs.index.duplicated(keep='first')]).dropna()

    # Convert end_time_input to a datetime.time object if it's a string
    if isinstance(end_time_input, str):
        # Parse the string to convert it into a time object
        end_time = datetime.strptime(end_time_input, '%H:%M:%S').time()
    elif isinstance(end_time_input, pd.Timestamp):
        # Convert Timestamp to time
        end_time = end_time_input.to_pydatetime().time()
    else:
        # Assuming end_time_input is already a datetime.time object
        end_time = end_time_input

    # Since start_time is the same as end_time, we don't need a separate calculation
    start_time = end_time

    # Adjust date index to datetime
    pd.to_datetime(df_w_rolling_z_scores_clean_w_diffs_deduped.index)

#     # Filtering the DataFrame based on the time window
#     res = df_w_rolling_z_scores_clean_w_diffs_deduped.between_time(start_time, end_time)
    
    return df_w_rolling_z_scores_clean_w_diffs_deduped


example = preprocess_data(raw_df, 10, 1, '11:30:00')
example.head(3)

,z_score_ESX_realized_1,z_score_ESX_realized_5,z_score_ESX_realized_10,z_score_ESX_realized_50,z_score_ESX_realized_100,z_score_ESX_move_1,z_score_ESX_move_5,z_score_ESX_move_10,z_score_ESX_move_50,z_score_ESX_move_100,...,z_score_ESPX_atm_5_shift_1,z_score_ESPX_atm_25_shift_1,z_score_ESPX_atm_50_shift_1,z_score_GBL_atm_5_shift_1,z_score_GBL_atm_25_shift_1,z_score_GBL_atm_50_shift_1,z_score_TN10_atm_5_shift_1,z_score_TN10_atm_25_shift_1,z_score_TN10_atm_50_shift_1,z_score_ESX_atm_50_shift_1
Date,,,,,,,,,,,,,,,,,,,,,
2014-05-27 10:15:00,-1.118922,-0.172561,-0.905108,-0.933323,-1.431936,1.014246,0.915370,0.629318,1.284532,1.375991,...,-0.273055,-0.259397,-0.256748,0.487927,-2.507596,-3.099998,0.011895,0.077292,0.228700,0.445326
2014-05-27 10:30:00,-1.423963,-0.160265,-0.601564,-1.052260,-1.124035,1.386884,0.407252,0.038958,1.130937,0.951935,...,-1.662665,-3.089852,-0.171810,-0.895556,-0.992782,-1.398980,-0.195542,-0.211285,-0.253763,-0.341859
2014-05-27 10:45:00,-0.981471,-0.177822,0.838807,-1.972919,-0.962251,1.218014,1.531919,1.092676,1.861524,1.451896,...,-3.094285,-0.540713,1.563597,-13.957096,-188.673855,2.400967,-0.123170,-0.142724,-0.158981,-0.132300


In [5]:
"""
NG:  My priors that determined my predictors were
(1)  The waiting curve for statarb price change signals is between 15 and 180 minutes (1 to 30 periods)
(2)  Intraday seasonality in price signals exist because there are patterns in when information is
     released. These are driven by schedules in macroeconomic announcements, exchange mechanics in liquidity
     and other similar factors.
(3)  GBL and TN10 were correlated, but GBL captures a more European asset, so I only looked at GBL.
(4)  I examined idiosyncratic changes in volatility metrics and moves. 
(5)  I would be interested in decomposing moves into positive and negative moves since I think the magnitude
     of effects would be different (with negative recent moves above the norm contributing to larger changes in
     interest for hedging via index put options). 
"""

predictors = [
        'GBL_atm_50_idio_var',
        'GBL_atm_5_idio_var_ratio_to_atm_50_idio_var',
        'ESX_move_1_ratio_to_move_50',
        'ESX_move_100_ratio_to_ESPX_move_100',          
]

import pandas as pd

def extract_dates_from_df(df, start_datetime_str):
    """
    Extracts rows from a DataFrame starting from a specified datetime.
    
    Parameters:
    df (pd.DataFrame): The DataFrame containing a DateTime index.
    start_datetime_str (str): The start datetime in string format (YYYY-MM-DD HH:MM:SS).
    
    Returns:
    pd.DataFrame: Filtered DataFrame starting from the specified datetime.
    """

    # Convert the start_datetime_str to a datetime object
    start_datetime = pd.to_datetime(start_datetime_str)

    # Filter the DataFrame based on the start datetime
    filtered_df = df[df.index >= start_datetime].index

    return filtered_df

start_datetime = '2014-08-15 09:00:00'
datetime_period_ends = extract_dates_from_df(raw_df, start_datetime)

def get_rolling_window(df, end_date_time_str, window_size='max'):
    """
    Creates a rolling window subset of the DataFrame up to a specified end datetime.

    Parameters:
    df (pd.DataFrame): DataFrame from which to create the rolling window.
    end_date_time_str (str): End datetime as a string.
    window_size (int or 'max', optional): Size of the rolling window in terms of number of rows.
                                          If 'max', includes all rows up to end_date_time_str.

    Returns:
    pd.DataFrame: Rolling window subset of the DataFrame.
    """
    print("Starting get_rolling_window...")

    # Convert end_date_time_str to a datetime object
    end_datetime = pd.to_datetime(end_date_time_str)
    print("End datetime:", end_datetime)

    # Trim the DataFrame to remove rows after end_datetime
    df_trimmed = df[df.index <= end_datetime]

    # Apply window size if specified and not 'max'
    if window_size != 'max':
        window_size = int(window_size)  # Ensure window_size is an integer
        if len(df_trimmed) > window_size:
            df_trimmed = df_trimmed.iloc[-window_size:]

    print("Rolling window selected with", len(df_trimmed), "rows.")
    return df_trimmed




print(datetime_period_ends)

DatetimeIndex(['2014-08-15 09:00:00', '2014-08-15 09:15:00',
               '2014-08-15 09:30:00', '2014-08-15 09:45:00',
               '2014-08-15 10:00:00', '2014-08-15 10:15:00',
               '2014-08-15 10:30:00', '2014-08-15 10:45:00',
               '2014-08-15 11:00:00', '2014-08-15 11:15:00',
               ...
               '2015-12-30 15:00:00', '2015-12-30 15:15:00',
               '2015-12-30 15:30:00', '2015-12-30 15:45:00',
               '2015-12-30 16:00:00', '2015-12-30 16:15:00',
               '2015-12-30 16:30:00', '2015-12-30 16:45:00',
               '2015-12-30 17:00:00', '2015-12-30 17:15:00'],
              dtype='datetime64[ns]', name='Date', length=11589, freq=None)


In [6]:
"""
NG:  My priors that determined the nature of the model:
(1)  Recent data contains more information so I weighted data based on a half-life tranformation.
(2)  I was asked to avoid any machine learning techniques so I used an OLS-based model with only 1
     predictor. 
(3)  I used a Newey-West HAC adjustment to adjust standard errors for heteroskedasticity and 
     autocorrelation. Because I am predicting each time of day separately, I wasn't sure what to put for
     the maxlags autocorrelation parameter, but I chose 5 somewhat arbitrarily.
(4)  Based on general financial markets priors, I forced the intercept coefficient to be zero.
"""

def perform_weighted_regression(X, y, half_life):
    """
    Performs weighted linear regression with a specific half-life and lag for the covariance matrix.
    
    Parameters:
    X (pd.Series or pd.DataFrame): Independent variables for regression.
    y (pd.Series): Dependent variable for regression.
    half_life (float): Half-life for weighting observations.
    lag (int): Lag to be used in HAC covariance matrix.
    
    Returns:
    RegressionResults: The fitted model's results.
    """    
    if isinstance(X, pd.Series):
        X = X.to_frame()
    if isinstance(y, pd.Series):
        y = y.to_frame()

    initial_data_count = len(X)
    X = handle_missing_data(X)
    y = handle_missing_data(y)

    # Print the count of data points removed
    removed_data_count = initial_data_count - len(X)
    print(f"Data points removed due to missing data: {removed_data_count}")

    # Align the indices of X and y
    X, y = X.align(y, join='inner', axis=0)

    weights = np.exp(-np.log(2) / half_life * np.arange(len(X)))
    model = WLS(y, sm.add_constant(X), weights=weights)
    results = model.fit(cov_type='HAC', cov_kwds={'maxlags': 5})

    # Return results along with the count of used data points
    return results, len(X)


In [7]:
"""
NG:  
(1)  I ran Optuna to optimize the n_period_change in metrics and halflife. I focused on these metrics to reflect the prior that
     volatility has regimes that change at a changing rate.
(2)  It was hard to get high R-squared values so I set the objective function to be out of sample
     R-squared if t-stat was high. This implicitly was to adjust for a Bonferroni-like correction factor 
     given that I am optimizing across a large permutation space even after reducing it based on priors.
(3)  I used a time-series based cross-validation Python library both within Optuna's objective function
     but I also reserved a testing set untouched by Optuna to reduce overfitting. 
"""

def objective(trial, df, predictor, end_datetime, tscv, n_splits, lookback_window_days):
    print(f"Starting optuna objective function call for trial: {trial.number}")
    local_df = df.copy()
    n_period_change = trial.suggest_int('n_period_change', 1, 10)    
    half_life = trial.suggest_int('half_life', 1, 15)       
    
    print(f"Suggested parameters - n_period_change: {n_period_change}, half_life: {half_life}")        
    df_filtered = preprocess_data(local_df, lookback_window_days, n_period_change, end_datetime)
    rolling_windows = get_rolling_window(df_filtered, end_datetime, lookback_window_days) 
    dependent_var_name = f"z_score_ESX_atm_50_idio_var_shift_{n_period_change}"
    predictor = f"z_score_{predictor}"
    
    # Initialize the score
    total_score = 0    
    
    # Iterate over each split using the provided tscv
    for train_index, test_index in tscv.split(df_filtered):
        train_fold, test_fold = df_filtered.iloc[train_index], df_filtered.iloc[test_index]

        # Perform the regression on the training fold
        regression_results, _ = perform_weighted_regression(train_fold[predictor], train_fold[dependent_var_name], half_life)
                
        # Evaluate on the test fold and accumulate the score
        test_score = custom_objective(regression_results, test_fold[dependent_var_name], test_fold[predictor])
        total_score += test_score

    # Average the score over all splits
    avg_score = total_score / n_splits
    
    print(f"Avg. score for trial: {avg_score}")    

    return avg_score

def custom_objective(regression_results, dependent_data, independent_data):
    t_stat = regression_results.tvalues.iloc[1]
    r_squared = regression_results.rsquared

    print(f"Calculating custom objective - T Statistic: {t_stat}, R-Squared: {r_squared}")
        
    # Determine the score based on t-statistic
    if abs(t_stat) < 2:
        print("t_stat too low, assigning zero for Optuna score")
        score = 0
    elif 2 <= abs(t_stat) < 4:
        print("t_stat OK, assigning 1/2 of r_squared value for Optuna score")
        score = 0.5 * r_squared
    else:  # abs(t_stat) >= 4
        print("t_stat looking good, assigning full r_squared value for Optuna score")
        score = r_squared

    # Return negative score for minimization
    print(f"Custom objective score: {score}. (We wil use negative of this value.)")    
    
    return -score

In [8]:
"""
NG:  
(1)  I converted predictors to z-score space calculated based on a 50 day lookback. Even though
     I used a half-life based weighting scheme in my OLS, this lookback also constrained the max amount
     of data that can be included in rolling models. 
(2)  Optuna natively supports multi-threading. I ran on the max number of cores. Depending on your computer, 
     this may crash Jupyter for you. Performance was stable at 2-3 cores with good performance too.
"""


def run_optuna_study(end_datetime, predictor_and_responder_df, predictor, responder):
#     """
#     Runs an Optuna optimization study to find the best parameters for weighted regression.
    
#     Parameters:
#     end_datetime (str): End datetime for the optimization period.
#     predictor_and_responder_df (pd.DataFrame): DataFrame containing predictors and responders.
    
#     Returns:
#     dict: Dictionary containing optimization results and statistics, including best parameters found by Optuna.
#     """       
    print(f"Starting Optuna optimization for specific datetime: {end_datetime} and predictor: {predictor}")    
    end_datetime = pd.to_datetime(end_datetime)
    local_df = predictor_and_responder_df.copy()
    n_splits = 3
    lookback_window_days = 50

    # Preprocess the data first but with no shift
    rolling_windows = get_rolling_window(local_df, end_datetime, "max")

    # Time Series Cross-Validation setup
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Create Optuna study
    study = optuna.create_study(direction='minimize', sampler=optuna.samplers.RandomSampler(), pruner=optuna.pruners.MedianPruner())

    # Find the training dataset using tscv
    for train_index, _ in tscv.split(rolling_windows):
        training_df = rolling_windows.iloc[train_index]
        break  # We only need the first split for training

    # Optimize to find best parameters
    study.optimize(lambda trial: objective(trial, training_df, predictor, end_datetime, tscv, n_splits, lookback_window_days), n_trials=50, n_jobs=-1)
    
    try:
        if len(study.trials) == 0 or study.best_trial is None:
            print("No successful trials were completed. Skipping this iteration.")
            return None
    except:
        pass

    # Extract the best parameters
    best_params = study.best_params
    best_n_period_change = best_params.get('n_period_change')
    best_half_life = best_params.get('half_life')
    rolling_windows_best_n_period_change = preprocess_data(local_df, lookback_window_days, best_n_period_change, end_datetime)  

    # Find the testing dataset using tscv
    for _, test_index in tscv.split(rolling_windows):
        testing_df = rolling_windows_best_n_period_change.iloc[test_index]
        
    adj_predictor = "z_score_" + predictor + f"_shift_{best_n_period_change}"
    adj_responder = "z_score_" + responder + f"_shift_{best_n_period_change}"

    # Apply best parameters and perform weighted regression on testing dataset
    results, used_data_points = perform_weighted_regression(testing_df[adj_predictor], testing_df[adj_responder], best_half_life)

    time_window_start = rolling_windows_best_n_period_change.index.min()
    time_window_end = rolling_windows_best_n_period_change.index.max()

    return {
        'time_window': {'start': time_window_start, 'end': time_window_end},
        'number_of_data_points_initial': len(local_df),
        'number_of_data_points_used': used_data_points,
        'optimal_parameters': {'n_period_change': best_n_period_change, 'half_life': best_half_life, 'lookback_window_days': lookback_window_days},
        'statistics': {'Asset': predictor, 'Beta': results.params.iloc[1], 'T_Statistic': results.tvalues.iloc[1], 'R_Squared': results.rsquared},
        'average_length': len(testing_df),
    }


In [ ]:
"""
NG:  
(1)  The model re-runs every 15 days even though it is trained over a longer window. This is mostly a 
     performance optimization given that I am running 3-fold validation twice and 50 permutations of Optuna
     across each 15 minute segment of the day separately. 
(2)  This code takes about 30 minutes or so to run and sometimes crashes Jupyter. Results are saved in pickle 
     for persistence. If the script fails, it will skip analysis it has already done by checking the pkl file 
     first so don't worry if it crashes. 
(3)  The data saved in the pkl file is out of sample data that the Optuna optimizer has not seen.
"""

import itertools 

def get_predictor_responder_df(df, predictor_name, responder_name='ESX_atm_50_idio_var'):
    """
    Extracts a DataFrame containing only the specified predictor and responder columns.

    Parameters:
    df (pd.DataFrame): The original DataFrame.
    predictor_name (str): The name of the predictor column.
    responder_name (str): The name of the responder column, defaulting to 'ESX_atm_50_idio_var'.

    Returns:
    pd.DataFrame: A DataFrame with only the specified predictor and responder columns.

    Raises:
    ValueError: If the specified columns do not exist in the DataFrame.
    """    
    # Check if the specified columns exist in the DataFrame
    if predictor_name not in df.columns or responder_name not in df.columns:
        raise ValueError("One or both of the specified column names do not exist in the DataFrame.")

    # Create a new DataFrame with the specified columns and the original index
    new_df = df[[predictor_name, responder_name]]

    return new_df

# Define a function to load existing results
def load_existing_results(filename):
    if os.path.exists(filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)
    return {}

def save_study_results(study_results):
    with open('optuna_study_results_v2.pkl', 'wb') as file:
        pickle.dump(study_results, file)


combinations = itertools.product(datetime_period_ends, predictors)

responder = 'ESX_atm_50_idio_var'

# Load existing results
study_results = load_existing_results('optuna_study_results_v2.pkl')

# study_results = {}

# Define the time window in days
time_window_days = 15

# Loop through combinations
for specific_datetime, predictor in combinations:
    # Extract the time component of the specific datetime
    specific_time = specific_datetime.time()

    # Flag to check if a similar study exists within the last 10 days
    study_exists_within_time_window = False

    # Filter the keys to consider only those that are tuples
    tuple_keys = [key for key in study_results.keys() if isinstance(key, tuple)]

    # Check for existing studies with the same time component
    for (existing_datetime, existing_predictor) in tuple_keys:
        if existing_datetime.time() == specific_time and existing_predictor == predictor:
            # Check if the study date is within the specified time window from the specific datetime
            if abs((specific_datetime.date() - existing_datetime.date()).days) <= time_window_days:
                study_exists_within_time_window = True
                break

    if not study_exists_within_time_window:
        print(f"Proceding with analysis for specific datetime {specific_datetime} and predictor {predictor}. No similar study was completed within the last {time_window_days} days.")        
        try:
            predictor_responder_df = get_predictor_responder_df(clean_df, predictor)
            result = run_optuna_study(specific_datetime, predictor_responder_df, predictor, responder)
            study_results[(specific_datetime, predictor)] = result

            # Print formatted results for this combination
            if result:  # Check if result is not None
                print_formatted_results(result, specific_datetime, predictor)
        except Exception as e:
            print(f"Error occurred during analysis for {specific_datetime} and {predictor}: {e}")
        
        # Save updated results to file for persistence after each iteration
        save_study_results(study_results)

    else:
        print(f"Skipping analysis for specific datetime {specific_datetime} and predictor {predictor} ...")


def print_formatted_results(result, specific_datetime, predictor):
    print(f"\nOUT OF SAMPLE RESULTS. Specific datetime {specific_datetime} and asset {predictor}:")
    print(f"  Time Window: {result['time_window']['start']} to {result['time_window']['end']}")                                
    print(f"  Time Window: {result['time_window']['start']} to {result['time_window']['end']}")
    print(f"  Number of Data Points: {result['number_of_data_points_used']}")
    print(f"  Optimal Parameters: n_period_change = {result['optimal_parameters']['n_period_change']}, half_life = {result['optimal_parameters']['half_life']}")
    print(f"  Statistics:")
    print(f"    Beta: {result['statistics']['Beta']}")
    print(f"    T Statistic: {result['statistics']['T_Statistic']}")
    print(f"    R-Squared: {result['statistics']['R_Squared']}")
    print(f"  Average Length: {result['average_length']}")

In [23]:
def prepare_data_for_tables(time_windows_results):
    aggregated_data = {}

    for (timestamp, asset), data in time_windows_results.items():
        if 'statistics' in data:
            month_year = timestamp.strftime("%b-%Y")
            hour = timestamp.strftime("%H:00")
            t_stat = data['statistics'].get('T_Statistic', 0)
            r_squared = data['statistics'].get('R_Squared', 0)

            key = f"{asset}_avg"
            aggregated_data.setdefault(hour, {}).setdefault(month_year, {}).update({
                f"{key}_t_stat": t_stat,
                f"{key}_r_squared": r_squared
            })

    return aggregated_data

def create_tables(aggregated_data):
    tables = {}

    for hour, month_data in aggregated_data.items():
        table_data = []
        for month_year, metrics in month_data.items():
            row = {'Month-Year': month_year}
            for key, value in metrics.items():
                if key != 'Best Metric':
                    row[key] = value
                else:
                    # Format best metric data
                    best_metric_info = metrics['Best Metric']
                    row['best_metric'] = f"{best_metric_info['Asset']} {best_metric_info['R-Squared']}\n({best_metric_info['Parameters']})"

            table_data.append(row)

        df = pd.DataFrame(table_data)
        df.set_index('Month-Year', inplace=True)
        tables[hour] = df

    return tables

aggregated_data = prepare_data_for_tables(time_windows_results)
tables = create_tables(aggregated_data)
tables

{'09:00':             GBL_atm_50_idio_var_avg_t_stat  GBL_atm_50_idio_var_avg_r_squared  \
 Month-Year                                                                      
 Aug-2014                         -5.281980                       5.943777e-01   
 Sep-2014                          0.196254                       5.560165e-05   
 Oct-2014                          2.468353                       7.084036e-02   
 Nov-2014                          0.186591                       2.726826e-03   
 Dec-2014                         -4.112607                       4.098406e-01   
 Jan-2015                         -0.211300                       2.399547e-03   
 Feb-2015                          0.621683                       3.543192e-03   
 Mar-2015                          1.032232                       8.106826e-02   
 Apr-2015                         29.423041                       7.869739e-01   
 May-2015                         -0.005177                       9.690614e-07   
 Jun-20

In [25]:
from IPython.display import HTML
from bs4 import BeautifulSoup

def display_table_for_hour(tables, hour, col_width=100):
    if hour in tables:
        df = tables[hour].copy()

        # Format numbers to 2 decimal places
        format_dict = {col: "{:.2f}" for col in df.select_dtypes(include=['float', 'float64']).columns}
        df_styled = df.style.format(format_dict).set_table_attributes('style="font-size: 12px; border: 1px solid black;"')

        # Convert to HTML to enforce column width and word wrap for cells
        html = df_styled.to_html()

        # Use BeautifulSoup to parse HTML and manipulate table headers
        soup = BeautifulSoup(html, 'html.parser')
        headers = soup.find_all('th')
        wrap_length = 10  # Adjust as needed

        # Ensure headers are not removed
        for header in headers:
            header_text = header.text
            header.clear()  # Clear the original text
            # Wrap text at the specified length and insert as HTML
            wrapped_header = '<br>'.join(header_text[i:i+wrap_length] for i in range(0, len(header_text), wrap_length))
            header.append(BeautifulSoup(wrapped_header, 'html.parser'))

        # Apply custom styles to headers for word wrap
        th_styles = {
            'text-align': 'center',
            'white-space': 'normal',
            'word-wrap': 'break-word',
            'width': f'{col_width}px',
            'border': '1px solid black'
        }
        for th in soup.find_all('th'):
            for style, value in th_styles.items():
                th[style] = value

        # Apply styles to body cells for borders
        td_styles = {
            'border': '1px solid black'
        }
        for td in soup.find_all('td'):
            for style, value in td_styles.items():
                td[style] = value

        # Display the modified HTML
        display(HTML(str(soup)))
    else:
        print(f"No data available for {hour}.")

def display_all_tables(tables, col_width=50):
    for hour in sorted(tables.keys()):
        print(f"Displaying table for {hour}")
        display_table_for_hour(tables, '09:00', col_width)
        print("\n")  # Adds a new line for better separation

# Example usage: display all tables for all hours
display_all_tables(tables)


Displaying table for 09:00


,GBL_atm_50_idio_var_avg_t_stat,GBL_atm_50_idio_var_avg_r_squared,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_t_stat,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_r_squared,ESX_move_1_ratio_to_move_50_avg_t_stat,ESX_move_1_ratio_to_move_50_avg_r_squared,ESX_move_100_ratio_to_ESPX_move_100_avg_t_stat,ESX_move_100_ratio_to_ESPX_move_100_avg_r_squared
Month-Year,,,,,,,,
Aug-2014,-5.28,0.59,2.33,0.34,-0.71,0.00,-10.61,0.35
Sep-2014,0.20,0.00,-4.07,0.51,0.54,0.00,0.61,0.00
Oct-2014,2.47,0.07,-1.41,0.06,-1.03,0.08,0.34,0.00
Nov-2014,0.19,0.00,0.53,0.00,0.60,0.02,-10.87,0.85
Dec-2014,-4.11,0.41,-3.26,0.35,-0.94,0.13,-2.23,0.35
Jan-2015,-0.21,0.00,2.68,0.27,-2.18,0.02,-9.58,0.59
Feb-2015,0.62,0.00,-1.11,0.00,-2.24,0.24,0.99,0.00
Mar-2015,1.03,0.08,2.07,0.07,5.86,0.78,-10.72,0.77
Apr-2015,29.42,0.79,8.78,0.62,-5.15,0.64,0.15,0.00




Displaying table for 10:00


,GBL_atm_50_idio_var_avg_t_stat,GBL_atm_50_idio_var_avg_r_squared,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_t_stat,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_r_squared,ESX_move_1_ratio_to_move_50_avg_t_stat,ESX_move_1_ratio_to_move_50_avg_r_squared,ESX_move_100_ratio_to_ESPX_move_100_avg_t_stat,ESX_move_100_ratio_to_ESPX_move_100_avg_r_squared
Month-Year,,,,,,,,
Aug-2014,-5.28,0.59,2.33,0.34,-0.71,0.00,-10.61,0.35
Sep-2014,0.20,0.00,-4.07,0.51,0.54,0.00,0.61,0.00
Oct-2014,2.47,0.07,-1.41,0.06,-1.03,0.08,0.34,0.00
Nov-2014,0.19,0.00,0.53,0.00,0.60,0.02,-10.87,0.85
Dec-2014,-4.11,0.41,-3.26,0.35,-0.94,0.13,-2.23,0.35
Jan-2015,-0.21,0.00,2.68,0.27,-2.18,0.02,-9.58,0.59
Feb-2015,0.62,0.00,-1.11,0.00,-2.24,0.24,0.99,0.00
Mar-2015,1.03,0.08,2.07,0.07,5.86,0.78,-10.72,0.77
Apr-2015,29.42,0.79,8.78,0.62,-5.15,0.64,0.15,0.00




Displaying table for 11:00


...

,GBL_atm_50_idio_var_avg_t_stat,GBL_atm_50_idio_var_avg_r_squared,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_t_stat,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_r_squared,ESX_move_1_ratio_to_move_50_avg_t_stat,ESX_move_1_ratio_to_move_50_avg_r_squared,ESX_move_100_ratio_to_ESPX_move_100_avg_t_stat,ESX_move_100_ratio_to_ESPX_move_100_avg_r_squared
Month-Year,,,,,,,,
Aug-2014,-5.28,0.59,2.33,0.34,-0.71,0.00,-10.61,0.35
Sep-2014,0.20,0.00,-4.07,0.51,0.54,0.00,0.61,0.00
Oct-2014,2.47,0.07,-1.41,0.06,-1.03,0.08,0.34,0.00
Nov-2014,0.19,0.00,0.53,0.00,0.60,0.02,-10.87,0.85
Dec-2014,-4.11,0.41,-3.26,0.35,-0.94,0.13,-2.23,0.35
Jan-2015,-0.21,0.00,2.68,0.27,-2.18,0.02,-9.58,0.59
Feb-2015,0.62,0.00,-1.11,0.00,-2.24,0.24,0.99,0.00
Mar-2015,1.03,0.08,2.07,0.07,5.86,0.78,-10.72,0.77
Apr-2015,29.42,0.79,8.78,0.62,-5.15,0.64,0.15,0.00




Displaying table for 16:00


,GBL_atm_50_idio_var_avg_t_stat,GBL_atm_50_idio_var_avg_r_squared,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_t_stat,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_r_squared,ESX_move_1_ratio_to_move_50_avg_t_stat,ESX_move_1_ratio_to_move_50_avg_r_squared,ESX_move_100_ratio_to_ESPX_move_100_avg_t_stat,ESX_move_100_ratio_to_ESPX_move_100_avg_r_squared
Month-Year,,,,,,,,
Aug-2014,-5.28,0.59,2.33,0.34,-0.71,0.00,-10.61,0.35
Sep-2014,0.20,0.00,-4.07,0.51,0.54,0.00,0.61,0.00
Oct-2014,2.47,0.07,-1.41,0.06,-1.03,0.08,0.34,0.00
Nov-2014,0.19,0.00,0.53,0.00,0.60,0.02,-10.87,0.85
Dec-2014,-4.11,0.41,-3.26,0.35,-0.94,0.13,-2.23,0.35
Jan-2015,-0.21,0.00,2.68,0.27,-2.18,0.02,-9.58,0.59
Feb-2015,0.62,0.00,-1.11,0.00,-2.24,0.24,0.99,0.00
Mar-2015,1.03,0.08,2.07,0.07,5.86,0.78,-10.72,0.77
Apr-2015,29.42,0.79,8.78,0.62,-5.15,0.64,0.15,0.00




Displaying table for 17:00


,GBL_atm_50_idio_var_avg_t_stat,GBL_atm_50_idio_var_avg_r_squared,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_t_stat,GBL_atm_5_idio_var_ratio_to_atm_50_idio_var_avg_r_squared,ESX_move_1_ratio_to_move_50_avg_t_stat,ESX_move_1_ratio_to_move_50_avg_r_squared,ESX_move_100_ratio_to_ESPX_move_100_avg_t_stat,ESX_move_100_ratio_to_ESPX_move_100_avg_r_squared
Month-Year,,,,,,,,
Aug-2014,-5.28,0.59,2.33,0.34,-0.71,0.00,-10.61,0.35
Sep-2014,0.20,0.00,-4.07,0.51,0.54,0.00,0.61,0.00
Oct-2014,2.47,0.07,-1.41,0.06,-1.03,0.08,0.34,0.00
Nov-2014,0.19,0.00,0.53,0.00,0.60,0.02,-10.87,0.85
Dec-2014,-4.11,0.41,-3.26,0.35,-0.94,0.13,-2.23,0.35
Jan-2015,-0.21,0.00,2.68,0.27,-2.18,0.02,-9.58,0.59
Feb-2015,0.62,0.00,-1.11,0.00,-2.24,0.24,0.99,0.00
Mar-2015,1.03,0.08,2.07,0.07,5.86,0.78,-10.72,0.77
Apr-2015,29.42,0.79,8.78,0.62,-5.15,0.64,0.15,0.00


In [26]:
"""
NG: Takeaways

ESX_move_100_ratio_to_ESPX_move_100 (unusual short-term changes in relative returns of ESX vs ESPX underlying) 
appears to be the best predictor of changes in ESX implied volatility across time of day and various 
market regimes, even on an idiosyncratic basis examined residual to a beta model from ESPX. 

Examining unusual short-term changes in nominal ESX move sizes (that is, not examined relative to ESPX) 
seem to be inferior to examining relative to ESPX moves. 

In certain market regimes, the change in GBL idiosyncratic implied volatility is a very strong predictors of
changes in ESX implied volatility, especially at certain times of day. Though it is far less consistent
of a predictor as ESPX in this regard. 

Calculating the z-score of typical ESX_move_100_ratio_to_ESPX_move_100 with a 50 day lookback, 
it appears that it is typically best to:
- Use relative changes over 60-90 minutes for the independent and dependent variable
- Use a 1-5 day half life in weighting datapoints for a regression. In other words, betas change
  frequently!

"""

def prepare_esx_data_for_tables(time_windows_results):
    aggregated_data = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

    for (timestamp, predictor), data in time_windows_results.items():
        if 'ESX_move_100_ratio_to_ESPX_move_100' in predictor:
            # Extract parameters
            params = data.get('optimal_parameters', {})
            n_period_change = params.get('n_period_change', None)
            half_life = params.get('half_life', None)
            lookback_window_days = params.get('lookback_window_days', None)

            # Format the timestamp
            month_year = timestamp.strftime("%b-%Y")
            hour = timestamp.strftime("%H:00")

            # Append parameter values to lists in the dictionary
            if n_period_change is not None:
                aggregated_data[hour][month_year]['n_period_change'].append(n_period_change)
            if half_life is not None:
                aggregated_data[hour][month_year]['half_life'].append(half_life)
            if lookback_window_days is not None:
                aggregated_data[hour][month_year]['lookback_window_days'].append(lookback_window_days)

    return aggregated_data


def create_esx_tables(aggregated_data):
    tables = {}

    for hour, month_data in aggregated_data.items():
        table_data = []
        for month_year, params in month_data.items():
            row = {'Month-Year': month_year}
            for param, values in params.items():
                # Calculate the average if values is a list, otherwise just use the value
                if isinstance(values, list) and values:
                    average_value = np.mean(values)
                else:
                    average_value = values
                row[f"{param}_avg"] = average_value
            table_data.append(row)

        df = pd.DataFrame(table_data)
        df.set_index('Month-Year', inplace=True)
        tables[hour] = df

    return tables

# Prepare the data
aggregated_data_2 = prepare_esx_data_for_tables(time_windows_results)

# Create the tables
tables_2 = create_esx_tables(aggregated_data_2)
tables_2

{'09:00':             n_period_change_avg  half_life_avg  lookback_window_days_avg
 Month-Year                                                              
 Aug-2014                  5.500          1.000                      50.0
 Sep-2014                  4.000          1.250                      50.0
 Oct-2014                  4.375          2.750                      50.0
 Nov-2014                  5.750          4.625                      50.0
 Dec-2014                  4.750          2.750                      50.0
 Jan-2015                  6.000          1.250                      50.0
 Feb-2015                  4.125          1.000                      50.0
 Mar-2015                  6.375          1.000                      50.0
 Apr-2015                  5.625          3.125                      50.0
 May-2015                  4.375          2.625                      50.0
 Jun-2015                  4.000          1.125                      50.0
 Jul-2015                  6.

In [ ]:
"""
NG:  Next steps
(1)  Discuss priors with someone with domain knowledge of product, geography and exchange mechanics.
     Further constrain optimization search space based on better informed priors. 
     Consider treating positive and negative moves differently in model
(2)  Confirm statistical concepts with others with more knowledge in modeling volatility 
     Model the residuals via EGARCH or GARCH? Or use GARCH earlier in process? 
     Lag parameter for the HAC adjustment?
     Thoughts on methodology? (Too much use of optimization?)
(3)  Examine intraday and weekly schedules of macroeconmic announcements, market mechanics and liquidity
     profiles of different assets. Start by talking to a domain expert to constrain research space.
     Consider differentiating variance associated with scheduled macroeconomic events from other changes. 
(4)  Improvements to make analysis more actionable (graph betas over time; determine whether residuals
     to my best model are mean reverting and, if so, over what time period; graph my predicted actual
     implied volatility values to actuals; etc)
     
"""